To open this notebook in Google Colab and start coding, click on the Colab icon below.

<table style="border:2px solid orange" align="left">
    <td style="border:2px solid orange">
        <a target="_blank" href="https://colab.research.google.com/github/neuefische/ds-meetups/blob/main/03_Recommender_Introduction/02_collaborative_filtering.ipynb">
        <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
    </td>
</table>

In [ ]:
# run on colab
!pip install surprise

# Collaborative filtering

In this notebook you will learn about collaborative filtering and how to implement it with the surprise library. Collaborative filtering is a collective term for different recommendation algorithms based on user behavior. Those algorithm find users similar to each other based on their rating or clicking history. The interactions between users and items are stored in a so-called "user-item interactions matrix". These interactions can be explicit like actively giving ratings or implicit like click-data. In general there are two popular types of collaborative filtering approaches. The user-based filtering and the item-based filtering.

User-based filtering algorithms predict ratings based on the ratings from similar (in terms of rating) users.</br>
Item-based filtering algorithms predict ratings based on the ratings of similar (in terms of rating) items. Item-based models are especially used when you have way more users than items. Those models use average rating per item and not per user.

A typical example of a problem collaborative filtering is trying to solve is the following: We have users, who rated specific items but a lot of item were not rated yet. We then try to predict the missing ratings denoted by red fields in this example of a user-item rating matrix.

<p align = "center">
<img src = "./images/UserItemRatingMatrix.png">
</p>
<p align = "center">
Fig.1 - User-Item-Rating Matrix - icons are from Vecteezy.com
</p>

first we import the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from copy import copy
from surprise import Dataset
from surprise import Reader
from surprise import KNNWithMeans, SVD
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict

Let's load our rating data. It contains the necessary `user_id`, `item_id` and the `rating` users gave to the fish items. Additionally it has some nice-to-have information about the fish items. There are 500 users with 300 rated fishes each. 

In [ ]:
# Loading the dataset from github may take some minutes -> coffee time :)
df = pd.read_csv('https://raw.githubusercontent.com/neuefische/ds-meetups/recommender_aljoscha/03_recommender/user_item_ratings.csv')
df.head(3)

The Surprise library we want to use does not work with pandas DataFrames but with Dataset objects. So we need to create a Dataset object from our DataFrame. We also need to define the possible ratings with the Reader class.

In [ ]:
# defines possible ratings
reader = Reader(rating_scale=(1, 10))
# Loads Pandas dataframe
data = Dataset.load_from_df(df[["user_id", "item_id", "rating"]], reader)

In order to validate our models we need to split our data into a trainset, which we will use to train our models. And a testset to validate the ability of our models to predict on unseen data. 

In [ ]:
trainset, testset = train_test_split(data, test_size=0.25)

## K-Nearest-Neighbors

One of the most common models for collaborative filtering is the K-nearest neighbor algorithm (KNN). KNN is a non-prarmetric, lazy learning method. Lazy because it just stores the the data-points  without learning any kind of coefficient. To make predictions it calculates the "distance" between the target and every other instance, then it ranks the distances and returns the top K who are closest and therefore most similar to a given data point. Several ways exist to calculate the distances between the target and the other observations.</br>
As KNN's performance suffers from curse of dimensionality and e.g. euclidean distance is not optimal in high dimensions, cosine similarity is the most popular distance measure in terms of multi-dimensional data. Further description of the cosine similarity can be found in notebook 1. In thies notebook we will use the [KNNWithMeans](https://surprise.readthedocs.io/en/stable/knn_inspired.html) algorithm implemented in the surprise library. This algorithm is directly derived from KNN but also takes the mean ratings of each user into account. 

In [ ]:
sim_options = {
    "name": "cosine",   # Use Cosine-Similarity
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)
algo.fit(trainset)

predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

## Singular Value Decomposition

Another way to estimate the user ratings is Singular Value Decomposition (SVD). Singular value decomposition is a matrix decomposition, where we decompose a given m x n matrix $A$ into three matrices $U$, $\Sigma$ and $V$ with the dimensions m x r, r x r and n x r respectively. Here r is usually small compared to m and n. And $\Sigma$ is non zero only on the diagonal.

![](./images/SVD_USigmaV.png) 

By decomposing the matrix $A$ in this way, we drastically reduce the number of elements to be stored. Let $m = n = 100$ and $r = 5$ then the original matrix $A$ has $100 \times 100 = 10000$ elements. Whereas $U$ has $100 \times 5 = 500$, $\Sigma$ has $5$ (only diagonal elements) and $V$ again has $100 \times 5 = 500$ elements leading to a total of only $1005$ elements as opposed to the $10000$ elements of the original matrix $A$.

The rows of m of our user-item-rating matrix $A$ refer to the users and the n columns to the items (fishes in our case). The introduced latent factors r can be interpreted as characteristics of the users for $U$ and item characteristics for $V$. In our case they can tell us how much a user likes a certain `visual_effect` or how much a fish shows said `visual_effect` respectively. $\Sigma$ then models the importance of each `visual_effect`.

The actual implementation of the algorithm in the surprise library will be outlined here to show an example of how to find an optimal decomposition. It became popular due to Simon Funk and his contribution to the Netflix competition. Here the rating user u gives item i is denoted by $r_{ui}$ and will be estimated by

$$
\hat r_{ui} = \mu + b_u + b_i + q^T_i p_u
$$

where 

- (u,i): user-item pair
- $\mu$: average rating of all items
- $b_i$: average rating of item i minus $\mu$
- $b_u$: average rating given by user u minus $\mu$
- $p_u$: latent user factor vector
- $q_i$: latent item factor vector

The parameters $b_i$, $b_u$, $p_u$ and $q_i$ are determined by minimizing the regularized squared error:

$$
\sum_{r_{ui} \in R_{train}}(r_{ui} - \hat r_{ui})^2 + \lambda(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2)
$$

The minimization is performed by a simple stochastic gradient descent (parameters are updated iteratively):

$$
\begin{align*}
b_u &\leftarrow b_u + \gamma(e_{ui} - \lambda b_u)\\
b_i &\leftarrow b_i + \gamma(e_{ui} - \lambda b_i)\\
p_u &\leftarrow p_u + \gamma(e_{ui}*q_i - \lambda p_u)\\
q_i &\leftarrow q_i + \gamma(e_{ui}*p_u - \lambda q_i)
\end{align*}
$$

where $e_{ui} = r_{ui} - \hat r_{ui}$, $\gamma$ is the learning rate and $\lambda$ is the regularization parameter.

Now that we understand SVD let us train the algorithm on our train set.

In [ ]:
# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Let's have a look at the top 10 recommendations for a specific user. Though there is no implementation of this in surprise the documentation provides a function `get_top_n` that returns the top-N recommendations, if we provide the predictions of our model:

In [ ]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

What we will get is a list of ten tuples (item_id, estimated_rating). 

In [ ]:
top_10 = get_top_n(predictions, n=10)   # top 10 recommendations for each user
user_id = 201   # the user we want the recommendations for
top_10[user_id]  # ten best rated items for user id

let's make list of the top 10 item id's `top_iids`. And use it with the original fishes dataframe to get some characteristics of our recommended fishes. Apparently our user liked especially colorful fishes the most :) sorry for the spaghetti in line 2 xD

In [ ]:
top_iids = [id_est[0] for id_est in top_n[201]]
recommended_fishes = df.set_index('item_id').loc[top_iids][['name','fish_group','visual_effect']].drop_duplicates()
recommended_fishes

## Parameter Grid Search

The SVD algorithm in the surprise library comes with a lot of hyper parameters which influence the performance of the model. To search for optimal hyper parameters in machine learning Grid search is used. It performs an exhaustive search over a prior defined parameter space using cross-validation (hence the CV suffix). That means it will evaluate all of the possible parameter combinations of the search space in order to find and return the best combination.

This task, however, starts to become very time-consuming if there are many hyperparameters and the search space is huge. As you can see for cv= 3 (number of folds that will be evaluated) and for 3 parameters with 2 values, thus 8 combinations, the GridSearchCV runs 24 modelling steps in order to just come up with the best values for the two parameters.

Please feel free to try out different values and beat the default ones from our prediction above ;) and if you still have time improve your result even further!

In [ ]:
param_grid = {
    "n_epochs": [10, 20],       # The number of iteration of the SGD procedure. Default is 20.
    "lr_all": [0.002, 0.005],   # The learning rate for all parameters. Default is 0.005.
    "reg_all": [0.02, 0.04]     # The regularization term for all parameters. Default is 0.02.
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])